<a href="https://colab.research.google.com/github/farisazizy/dota2-733e-meta-analysis/blob/main/GG3_DA_GG3DAUP0032_MIDTERM_TEST_FILE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Farishadi Mufakkir Azizy - GG3DAUP0032

# Import Library

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import textwrap
import warnings

warnings.filterwarnings('ignore')

# Import Data

In [97]:
df = pd.read_csv('Current_Pro_meta.csv')
df = df.drop(df.columns[0], axis = 1)
df

,Name,Primary Attribute,Attack Type,Attack Range,Roles,Total Pro wins,Times Picked,Times Banned,Win Rate,Niche Hero?
0,Abaddon,all,Melee,150,"Support, Carry, Durable",22,46,108,47.83,False
1,Alchemist,str,Melee,150,"Carry, Support, Durable, Disabler, Initiator, ...",50,102,86,49.02,False
2,Ancient Apparition,int,Ranged,675,"Support, Disabler, Nuker",25,53,84,47.17,False
3,Anti-Mage,agi,Melee,150,"Carry, Escape, Nuker",76,175,636,43.43,False
4,Arc Warden,agi,Ranged,625,"Carry, Escape, Nuker",8,16,74,50.00,False
...,...,...,...,...,...,...,...,...,...,...
119,Windranger,all,Ranged,600,"Carry, Support, Disabler, Escape, Nuker",227,461,887,49.24,False
120,Winter Wyvern,all,Ranged,425,"Support, Disabler, Nuker",29,57,347,50.88,False
121,Witch Doctor,int,Ranged,550,"Support, Nuker, Disabler",38,72,20,52.78,True
122,Wraith King,str,Melee,150,"Carry, Support, Durable, Disabler, Initiator",38,68,24,55.88,True


In [103]:
df['Times Picked'].isnull().any()

False

In [98]:
df.to_csv('Current_Pro_meta_fixed.csv', index=False)

# Create specific table for every hero roles

## proceed to split

In [81]:
# Create a new DataFrame with each row containing one role per hero
roles_df = df.copy()

# If 'Times Picked' and 'Win Rate' are not numeric columns, you might need to convert them to numeric types.
roles_df['Times Picked'] = pd.to_numeric(roles_df['Times Picked'], errors='coerce')
roles_df['Win Rate'] = pd.to_numeric(roles_df['Win Rate'], errors='coerce')

# Split the roles into separate columns
roles_df['Roles'] = roles_df['Roles'].str.split(',')

# Use melt to reshape the DataFrame with one role per row for each hero
roles_df = roles_df.explode('Roles').reset_index(drop=True)

# Strip leading/trailing whitespaces from the roles
roles_df['Roles'] = roles_df['Roles'].str.strip()

# Sort the new DataFrame by 'Name' and 'Roles' to group the roles for each hero together
roles_df.sort_values(['Name', 'Roles'], inplace=True)

# Reset the index again after sorting to have a continuous index without duplicates
roles_df.reset_index(drop=True, inplace=True)

# Print the new table with each hero's roles split into separate rows
roles_df

,Name,Primary Attribute,Attack Type,Attack Range,Roles,Total Pro wins,Times Picked,Times Banned,Win Rate,Niche Hero?
0,Abaddon,all,Melee,150,Carry,22,46,108,47.83,False
1,Abaddon,all,Melee,150,Durable,22,46,108,47.83,False
2,Abaddon,all,Melee,150,Support,22,46,108,47.83,False
3,Alchemist,str,Melee,150,Carry,50,102,86,49.02,False
4,Alchemist,str,Melee,150,Disabler,50,102,86,49.02,False
...,...,...,...,...,...,...,...,...,...,...
476,Wraith King,str,Melee,150,Durable,38,68,24,55.88,True
477,Wraith King,str,Melee,150,Initiator,38,68,24,55.88,True
478,Wraith King,str,Melee,150,Support,38,68,24,55.88,True
479,Zeus,int,Ranged,380,Carry,70,145,232,48.28,False


In [82]:
roles_df.to_csv('roles.csv', index=False)

## Add to CSV per roles

In [83]:
for role, role_group in roles_df.groupby('Roles'):
    role_table = role_group[['Name', 'Roles', 'Total Pro wins', 'Times Picked', 'Times Banned', 'Win Rate', 'Niche Hero?']].copy()
    role_table.reset_index(drop=True, inplace=True)
    role_table.to_csv(f'roles_{role.lower()}.csv', index=False)

# Pick and Ban Analysis

# Win Rate Analysis